# Gray-Scott f-k Parameter Map Visualization (Google Colab版)

Interactive visualization of Gray-Scott reaction-diffusion patterns.
Click on any point in the f-k map to view the corresponding GIF animation.

## 📋 セットアップ手順

1. **必要なファイルをアップロード**
   - `fk_analysis.py` (このnotebookと同じディレクトリに配置)
   - `fk_data_cache.npz` (データキャッシュファイル、約4.2MB)
   - `gif/` フォルダ (GIFファイル、約7GB)

2. **Google Driveを使用する場合（推奨）**
   - Google Driveに`gif`フォルダをアップロード
   - 下記のセルでDriveをマウント

3. **Colabに直接アップロードする場合**
   - ファイル → フォルダをアップロード（小規模データ向け）

## 1. 必要なライブラリのインストール

In [1]:
# 必要なライブラリをインストール
!pip install -q ipywidgets ipympl pillow numpy matplotlib


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 2. Google Driveのマウント（GIFファイルをDriveに保存している場合）

In [2]:
# Google Driveをマウント
# 初回実行時は認証が必要です
from google.colab import drive
drive.mount('/content/drive')

# Drive内のgifフォルダのパスを設定
# 例: /content/drive/MyDrive/GrayScott/gif
GIF_DIR_DRIVE = '/content/drive/MyDrive/GrayScott/gif'  # ここを自分のパスに変更してください

# ローカルディレクトリにシンボリックリンクを作成（オプション）
import os
if os.path.exists(GIF_DIR_DRIVE):
    if not os.path.exists('/content/gif'):
        os.symlink(GIF_DIR_DRIVE, '/content/gif')
        print(f'Created symlink: /content/gif -> {GIF_DIR_DRIVE}')
    else:
        print('Symlink already exists')
else:
    print(f'Warning: {GIF_DIR_DRIVE} does not exist. Please check the path.')

ModuleNotFoundError: No module named 'google.colab'

## 3. ファイルのアップロード

以下のファイルをアップロードしてください：
- `fk_analysis.py`
- `fk_data_cache.npz`

In [ ]:
# ファイルアップロード用（fk_analysis.pyとfk_data_cache.npz）
from google.colab import files
import os

print("以下のファイルをアップロードしてください:")
print("1. fk_analysis.py")
print("2. fk_data_cache.npz")
print("\nアップロードを開始します...")

uploaded = files.upload()

# アップロードされたファイルを確認
for filename in uploaded.keys():
    print(f'Uploaded: {filename} ({len(uploaded[filename])} bytes)')

## 4. 環境設定とインポート

In [ ]:
# Colab用のmatplotlibバックエンド設定
# Colabでは%matplotlib widgetが使えないため、%matplotlib notebookを使用
try:
    %matplotlib notebook
    print('Using %matplotlib notebook')
except Exception:
    %matplotlib inline
    print('Using %matplotlib inline (click events may not work)')
    print('Note: For full interactivity, consider using %matplotlib notebook')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output, HTML
import ipywidgets as widgets
import os
import base64

# fk_analysisモジュールをインポート
import fk_analysis

print('All imports successful!')

## 5. データの読み込み

キャッシュファイル（`fk_data_cache.npz`）からデータを読み込みます。
初回実行時やキャッシュがない場合は、GIFファイルからデータを処理します（数分かかります）。

In [ ]:
# GIFディレクトリのパスを設定
# オプション1: Google Driveを使用する場合（上記でマウント済み）
GIF_DIR = '/content/gif'  # シンボリックリンク経由

# オプション2: Colabに直接アップロードした場合
# GIF_DIR = '/content/gif'  # アップロードしたフォルダのパス

# オプション3: キャッシュファイルのみ使用（GIFファイルがない場合）
# GIF_DIR = None  # この場合、キャッシュファイルのみから読み込みます

# データの読み込み
print(f'Loading data from cache or processing GIF files in: {GIF_DIR if GIF_DIR else "cache only"}')

if GIF_DIR and os.path.exists(GIF_DIR):
    # GIFディレクトリが存在する場合
    data = fk_analysis.load_or_process_data(gif_dir=GIF_DIR, use_cache=True, verbose=True)
else:
    # キャッシュファイルのみから読み込む
    print('GIF directory not found. Loading from cache only...')
    try:
        data = fk_analysis.load_cache('fk_data_cache.npz')
        print('Cache loaded successfully!')
    except Exception as e:
        print(f'Error loading cache: {e}')
        print('Please ensure fk_data_cache.npz is uploaded.')
        raise

f_values = data['f_values']
k_values = data['k_values']
variations = data['variations']
avail_paths = data['paths']

print(f'\nLoaded {len(f_values)} data points')
print(f'f range: [{f_values.min():.4f}, {f_values.max():.4f}]')
print(f'k range: [{k_values.min():.4f}, {k_values.max():.4f}]')

## 6. インタラクティブ可視化のセットアップ

In [ ]:
# GIFプレビュー用のウィジェットを作成
output_box = widgets.Output(layout=widgets.Layout(
    width='128px', height='128px', overflow='hidden', 
    border='1px solid #ccc', flex='0 0 auto', 
    padding='0px', margin='0px'
))

filename_label = widgets.HTML(
    value='<div style="width:128px; word-wrap:break-word; font-size:12px; text-align:center;">'
          'Click a point to preview</div>'
)

gif_container = widgets.VBox(
    [filename_label, output_box],
    layout=widgets.Layout(gap='4px')
)

print('Widgets created successfully!')

## 7. f-kマップの作成と表示

In [ ]:
# インタラクティブなf-kマップを作成
plt.ioff()
fig, ax = plt.subplots(figsize=(8, 6))

# 散布図（空間変化率で色分け）
sc = ax.scatter(k_values, f_values, c=variations, s=24, alpha=0.7, 
                picker=5, cmap='viridis')

ax.set_xlabel('k', fontsize=12)
ax.set_ylabel('f', fontsize=12)
ax.set_title('f-k Map: Spatial Variation (click to view GIF)', fontsize=14)
ax.grid(True, alpha=0.3)

# 理論的分岐曲線を追加
kmin, kmax = k_values.min(), k_values.max()
fmin, fmax = f_values.min(), f_values.max()

def plot_curve_simple(G, klim, flim, ax, color='crimson', linewidth=2, label=None, n=400):
    """分岐曲線をプロットする関数"""
    k = np.linspace(klim[0], klim[1], n)
    f = np.linspace(flim[0], flim[1], n)
    K, F = np.meshgrid(k, f)
    Z = G(F, K)
    if np.iscomplexobj(Z):
        Z = np.where(np.abs(np.imag(Z)) < 1e-10, np.real(Z), np.nan)
    else:
        Z = np.asarray(Z, dtype=float)
    ax.contour(K, F, Z, levels=[0.0], colors=color, linewidths=linewidth, alpha=0.8)
    if label:
        ax.plot([], [], color=color, linewidth=linewidth, label=label)

# Hopf分岐曲線
def G_Hopf(f, k):
    return 4*k - (f + np.sqrt(f*(f-4*(f+k)**2)))**2 / ((f + k)**2)

plot_curve_simple(G_Hopf, (kmin, kmax), (fmin, fmax), ax, 
                  color='green', linewidth=1.5, label='Hopf')

# 数値解境界
def G_NumSoln(f, k):
    return f - 4*f**2 - 8*f*k - 4*k**2

plot_curve_simple(G_NumSoln, (kmin, kmax), (fmin, fmax), ax, 
                  color='red', linewidth=1.5, label='NumSoln')

ax.legend(loc='upper right', fontsize=9)

# アスペクト比の設定
data_range_k = k_values.max() - k_values.min()
data_range_f = f_values.max() - f_values.min()
ax.set_aspect(data_range_k / (1.2 * data_range_f), adjustable='box')

fig.tight_layout(pad=2.0)

print('Figure created successfully!')

## 8. クリックイベントハンドラの設定

In [ ]:
# クリックイベントハンドラ
last_annotation = None

def on_pick(event):
    """マップ上の点がクリックされたときの処理"""
    global last_annotation
    
    # 前のアノテーションを削除
    if last_annotation is not None:
        try:
            last_annotation.remove()
        except:
            pass
        last_annotation = None

    ind = event.ind
    if len(ind) == 0:
        return
    
    idx = ind[0]
    f = float(f_values[idx])
    k = float(k_values[idx])
    var = float(variations[idx])

    # クリックされた点にアノテーションを追加
    try:
        last_annotation = ax.annotate(
            f"f={f:.4f}, k={k:.4f}\nvar={var:.1f}", (k, f),
            textcoords="offset points", xytext=(10, 10),
            bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.6)
        )
        fig.canvas.draw_idle()
    except Exception as e:
        print(f'Error creating annotation: {e}')

    # 対応するGIFファイルを探す
    gif_path = fk_analysis.find_gif_for_fk(f, k, f_values, k_values, avail_paths)
    
    if not gif_path:
        print(f'GIF not found for f={f:.4f}, k={k:.4f}')
        return
    
    # パスの調整（Drive経由の場合）
    if not os.path.exists(gif_path):
        # パスを調整してみる
        filename = os.path.basename(gif_path)
        # 複数の可能性のあるパスを試す
        possible_paths = [
            os.path.join('/content/gif', filename),
            os.path.join('/content/drive/MyDrive/GrayScott/gif', filename),
            os.path.join(GIF_DIR if 'GIF_DIR' in globals() else '/content/gif', filename)
        ]
        
        found = False
        for path in possible_paths:
            if os.path.exists(path):
                gif_path = path
                found = True
                break
        
        if not found:
            print(f'GIF not found: {filename}')
            print(f'Tried paths: {possible_paths}')
            return

    print(f'Displaying: {gif_path}')
    
    # ファイル名ラベルの更新
    filename = os.path.basename(gif_path)
    filename_label.value = (
        f'<div style="width:128px; word-wrap:break-word; font-size:12px; text-align:center;">'
        f'{filename}</div>'
    )
    
    # GIFを表示（base64エンコード）
    with output_box:
        clear_output(wait=True)
        try:
            with open(gif_path, 'rb') as f:
                gif_data = base64.b64encode(f.read()).decode('utf-8')
            display(HTML(
                f'<img src="data:image/gif;base64,{gif_data}" '
                f'width="128" height="128" '
                f'style="display:block; margin:0; padding:0; border:none; line-height:0;" />'
            ))
        except Exception as e:
            print(f'Error loading GIF: {e}')

# クリックイベントを接続
cid = fig.canvas.mpl_connect('pick_event', on_pick)
print('Click event handler connected!')

## 9. 可視化の表示

In [ ]:
# キャンバスとコンテナの設定
fig.canvas.layout = widgets.Layout(
    width='700px', height='600px', 
    border='1px solid #ccc', flex='0 0 auto'
)

container = widgets.HBox(
    [fig.canvas, gif_container],
    layout=widgets.Layout(
        align_items='center', 
        justify_content='flex-start', 
        gap='12px', 
        overflow='visible'
    )
)

# 表示
display(container)

print('\n✅ Setup complete! Click on any point in the map to view the corresponding GIF animation.')
print('\n💡 Tips:')
print('   - If click events don\'t work, try using %matplotlib notebook instead of inline')
print('   - Make sure the GIF directory path is correct')
print('   - Check that GIF files are accessible from the current path')

## 📝 トラブルシューティング

### クリックイベントが動作しない場合

1. **matplotlibバックエンドの確認**
   - `%matplotlib notebook`を使用しているか確認
   - カーネルを再起動してから再実行

2. **GIFファイルが見つからない場合**
   - Google Driveのパスが正しいか確認
   - シンボリックリンクが正しく作成されているか確認
   - `GIF_DIR`変数を正しく設定しているか確認

3. **メモリ不足エラー**
   - Colabのランタイムタイプを「高メモリ」に変更
   - 不要なセルをクリア

### ファイルのアップロード方法

**方法1: Google Drive（推奨、大容量ファイル向け）**
1. Google Driveに`gif`フォルダをアップロード
2. 上記のセルでDriveをマウント
3. パスを設定

**方法2: Colabに直接アップロード（小規模データ向け）**
1. ファイル → フォルダをアップロード
2. `/content/gif`に配置

**方法3: キャッシュファイルのみ使用**
1. `fk_data_cache.npz`のみをアップロード
2. GIFファイルなしで可視化のみ実行（GIF表示は不可）